In [201]:
import pandas as pd


In [206]:
f = open("p.txt","r")   
p = f.read()

# 将销售量低于2（这个参数可配置）的删除掉
num = 2
ti1 = pd.read_excel("ti1.xls")
ti2 = pd.read_excel("ti2.xls")
ti3 = pd.read_excel("../淘汰商品列表.xls")
vendor = pd.read_csv("../供应商列表.csv",encoding="gbk")

# ti1  进销存报表
# ti2  采购订单
# ti3  淘汰商品列表
# vendor  供应商列表


In [199]:
vendor

,供应商名称,商品编码,供应商旺旺号
0,UNIKON,13280,NaN
1,安吉全丰竹工艺品有限公司,15418,NaN
2,蔡永红,16138,sytonn
3,蔡永红,15098,sytonn
4,蔡永红,16174,sytonn
...,...,...,...
798,诸暨市杨劲袜厂,15076,yangjinwaye
799,诸暨市杨劲袜厂,15201,yangjinwaye
800,诸暨市叶昊袜厂,16022,xishutiantian1
801,子腾快递,打包用品,NaN


In [204]:
# 给提供多个产品的供应商加上标签 
vv2 = vendor[['供应商名称','商品编码']]
vv3 = vv2.groupby('商品编码').count()
vv4 =  pd.merge(vendor,vv3,how='inner',on='商品编码')
# vv4['供应商名称_x'].values.tolist()
# 判断是不是 == 1 是的话该列为+'(多供应商)'
def apply_vv(df):
    if  str(df['供应商名称_y']) != '1':
        return str(df['供应商名称_x'])+'(多供应商)'
    else:
        return str(df['供应商名称_x'])
    
vv4['供应商名称'] = vv4.apply(apply_vv , axis = 1)
vendor = vv4[['供应商名称','商品编码','供应商旺旺号']]
# vendor.drop_duplicates(subset=['供应商名称'], keep='first', inplace=True)
aa = vendor['供应商名称'].values.tolist()
aa

['UNIKON(多供应商)',
 '玖悦日本(多供应商)',
 '安吉全丰竹工艺品有限公司',
 '蔡永红(多供应商)',
 '王丙乾(多供应商)',
 '蔡永红',
 '蔡永红',
 '蔡永红(多供应商)',
 '缙云县卓越塑料制品厂(多供应商)',
 '温州沪深工艺礼品有限公司(多供应商)',
 '宜兴市宜城镇张根水五金店(多供应商)',
 '义乌市爱奇文具厂(多供应商)',
 '义乌市宝轩电子商务有限公司(多供应商)',
 '义乌市标顺饰品厂(多供应商)',
 '义乌市晨梦家居用品有限公司(多供应商)',
 '义乌市迪辰文化用品有限公司(多供应商)',
 '义乌市雷群贸易有限公司(多供应商)',
 '义乌市李汉斌日用品有限公司(多供应商)',
 '义乌市吕塑日用品厂(多供应商)',
 '义乌市荣灵电子商务商行(多供应商)',
 '义乌市荣灵日用百货有限公司(多供应商)',
 '义乌市诗茵电子商务有限公司(多供应商)',
 '义乌市腾熙贸易有限公司(多供应商)',
 '义乌市艺蓝文化用品有限公司(多供应商)',
 '义乌市佑木日用品商行限公司(多供应商)',
 '苍南缤纷文具有限公司(多供应商)',
 '义乌市诗茵电子商务有限公司(多供应商)',
 '苍南缤纷文具有限公司(多供应商)',
 '苍南优佳文具商行(多供应商)',
 '苍南缤纷文具有限公司',
 '苍南缤纷文具有限公司',
 '苍南缤纷文具有限公司(多供应商)',
 '苍南优佳文具商行(多供应商)',
 '苍南缤纷文具有限公司',
 '苍南缤纷文具有限公司(多供应商)',
 '苍南优佳文具商行(多供应商)',
 '苍南恩赐利文具有限公司',
 '苍南县臣安工艺品厂',
 '苍南县臣安工艺品厂',
 '苍南县臣安工艺品厂(多供应商)',
 '姑苏区摩漫城亚平日用品店(多供应商)',
 '义乌市迪辰文化用品有限公司(多供应商)',
 '义乌市诗茵电子商务有限公司(多供应商)',
 '苍南县臣安工艺品厂(多供应商)',
 '义乌市迪辰文化用品有限公司(多供应商)',
 '义乌市芬菲文化用品有限公司(多供应商)',
 '苍南县臣安工艺品厂',
 '苍南县臣安工艺品厂',
 '苍南县臣安工艺品厂',
 '苍南县臣安工艺品厂',
 '苍南县臣安工艺品厂',
 '苍南县臣安工艺品厂',
 '苍南县臣安工艺品厂',

In [120]:
print(ti1.shape)
# 去除空格之类
ti1['商品编码'] = ti1['商品编码'].str.replace('\t', '')
ti1['商品编码'] = ti1['商品编码'].str.replace(' ', '')
ti1['规格编码'] = ti1['规格编码'].str.replace('\t', '')
ti1['规格编码'] = ti1['规格编码'].str.replace(' ', '')

ti1.dropna(axis=0,subset=["规格编码", "商品编码"])   # 去掉nan
ti1.shape

(9957, 46)


(9957, 46)

In [121]:
# 将进销存报表中各个商品的各个规格（SKU）的销售数量*参数p 与 该SKU的期末库存数量进行比较 
# 取 销售数量*参数p >= 期末库存数量 
t1 = ti1[["商品编码","商品名称","规格编码","规格名称","销售数量","期末库存数量"]]
to1 = t1[int(p) * t1['销售数量'] >= t1['期末库存数量']]
to1


,商品编码,商品名称,规格编码,规格名称,销售数量,期末库存数量
0,12109,电脑左/右侧留言板,12109-001,"天使右侧款,",256,176
1,12109,电脑左/右侧留言板,12109-002,"天使左侧款,",194,185
2,12109,电脑左/右侧留言板,12109-003,"猫头鹰左侧款,",460,130
3,12109,电脑左/右侧留言板,12109-004,"猫头鹰右侧款,",643,262
4,12109,电脑左/右侧留言板,12109-005,"透明右侧款,",521,322
...,...,...,...,...,...,...
9848,19557,双面刺绣渔夫帽,19557-002,笑脸,5,5
9918,2082,退货,2082,NaN,19,-575
9928,T19333,推拉式储米箱,T19333-005,棕色-小号,4,4
9940,T19442,(T)三角金属书立架,T19442-002,"玫瑰金,26*18*17.5cm",2,2


In [122]:
# 并将同商品编码的符合上述条件的各SKU的销售数量累加，成为该商品的总销量。
a =  to1[["商品编码","销售数量"]]
b = a.groupby(['商品编码']).sum()
b


,销售数量
商品编码,
001,65
12010,0
12049,4
12109,4984
12141,516
...,...
T19518,1
T19520,42
T19526,1


In [123]:
# 合并
toaa = pd.merge(to1,b,how='inner',on='商品编码')
# 重命名 销售数量_x 销售数量_y --商品总销量
toaa.rename(columns={'销售数量_y': '商品总销量','销售数量_x':'销售数量'}, inplace=True)

toaa

,商品编码,商品名称,规格编码,规格名称,销售数量,期末库存数量,商品总销量
0,12109,电脑左/右侧留言板,12109-001,"天使右侧款,",256,176,4984
1,12109,电脑左/右侧留言板,12109-002,"天使左侧款,",194,185,4984
2,12109,电脑左/右侧留言板,12109-003,"猫头鹰左侧款,",460,130,4984
3,12109,电脑左/右侧留言板,12109-004,"猫头鹰右侧款,",643,262,4984
4,12109,电脑左/右侧留言板,12109-005,"透明右侧款,",521,322,4984
...,...,...,...,...,...,...,...
3704,19557,双面刺绣渔夫帽,19557-002,笑脸,5,5,5
3705,2082,退货,2082,NaN,19,-575,19
3706,T19333,推拉式储米箱,T19333-005,棕色-小号,4,4,4
3707,T19442,(T)三角金属书立架,T19442-002,"玫瑰金,26*18*17.5cm",2,2,2


In [124]:
# （需要注意的是，会出现同一商品中有一个或几个SKU并没有满足上述条件，此时这些SKU的销售量并没有计入总销售量）
# ，在按照各商品总销量进行降序排列就形成 to1.csv
# 排序
to1 = toaa.sort_values(['商品总销量','销售数量'], ascending=[False,False])
to1 = to1.reset_index(drop=True)
to1 = to1[to1['销售数量'] >= num]
to1
############################## hava to1 ##############################################

,商品编码,商品名称,规格编码,规格名称,销售数量,期末库存数量,商品总销量
0,19013,小象冒泡居家凉拖鞋,19013-005,紫色38-39码,5699,2613,58044
1,19013,小象冒泡居家凉拖鞋,19013-012,浅绿38-39码,5259,1898,58044
2,19013,小象冒泡居家凉拖鞋,19013-002,粉色38-39码,5256,2496,58044
3,19013,小象冒泡居家凉拖鞋,19013-015,灰色42-43码,4302,2760,58044
4,19013,小象冒泡居家凉拖鞋,19013-001,粉色36-37码,3663,1871,58044
...,...,...,...,...,...,...,...
3439,18315,恶搞圣旨便签本?,18315-003,爱卿平身,2,1,2
3440,17051,阿狸图案隐形船袜,17051-005,深粉色狐狸,2,-2,2
3441,18311,中国风塑料圆扇,18311-003,傅朕旨意,2,1,2
3442,17298,水果半透明、超大笔袋,17298-016,倒梯形-粉色桃子,2,2,2


In [93]:
# 在to1 的基础上删除掉采购未到货的商品SKU（从采购订单报表中可以得到） 。
# 再删除掉淘汰商品编码（从淘汰商品列表中可以得到）。形成to2.csv

In [125]:
to_2 = to1[-to1['规格编码'].isin(ti2['商品规格编码'])]   #   - not in 

to__2 = to_2[-to_2['商品编码'].isin(ti3['淘汰商品'])] 
to2 = to__2[-to__2['规格编码'].isin(ti3['淘汰商品'])] 
to2

,商品编码,商品名称,规格编码,规格名称,销售数量,期末库存数量,商品总销量
0,19013,小象冒泡居家凉拖鞋,19013-005,紫色38-39码,5699,2613,58044
1,19013,小象冒泡居家凉拖鞋,19013-012,浅绿38-39码,5259,1898,58044
2,19013,小象冒泡居家凉拖鞋,19013-002,粉色38-39码,5256,2496,58044
3,19013,小象冒泡居家凉拖鞋,19013-015,灰色42-43码,4302,2760,58044
4,19013,小象冒泡居家凉拖鞋,19013-001,粉色36-37码,3663,1871,58044
...,...,...,...,...,...,...,...
3439,18315,恶搞圣旨便签本?,18315-003,爱卿平身,2,1,2
3440,17051,阿狸图案隐形船袜,17051-005,深粉色狐狸,2,-2,2
3441,18311,中国风塑料圆扇,18311-003,傅朕旨意,2,1,2
3442,17298,水果半透明、超大笔袋,17298-016,倒梯形-粉色桃子,2,2,2


In [95]:
# 基于to2.csv 根据商品编码匹配 供应商列表中的信息，
# 提取出供应商名称及旺旺号（也可能没有，则设置为“无供应商信息，商品编码xxxxx”），
# 并统计出该供应商各商品编码中，销售量最大的一个商品的数量（无供应商信息，则销量就是该商品的销售量）。
# 将供应商信息，销售量最大的商品的数量这两个信息与to2.csv合并，再进行排版规格化，最终形成to3.csv

In [126]:
# 合并
to3_tem = pd.merge(to2,vendor,how='left',on='商品编码')

supplier_is_null = pd.isna(to3_tem['供应商名称']) # 返回True和False
to3_tem["exist_supplier"] = supplier_is_null

to3_tem["供应商名称"] = to3_tem["供应商名称"].fillna('无供应商信息-商品编码：'+to3_tem["商品编码"])
to3_tem["供应商旺旺号"] = to3_tem["供应商旺旺号"].fillna('无')

# len(to3_tem['供应商旺旺号'].unique())  54
to3_tem

,商品编码,商品名称,规格编码,规格名称,销售数量,期末库存数量,商品总销量,供应商名称,供应商旺旺号,exist_supplier
0,19013,小象冒泡居家凉拖鞋,19013-005,紫色38-39码,5699,2613,58044,无供应商信息-商品编码：19013,无,True
1,19013,小象冒泡居家凉拖鞋,19013-012,浅绿38-39码,5259,1898,58044,无供应商信息-商品编码：19013,无,True
2,19013,小象冒泡居家凉拖鞋,19013-002,粉色38-39码,5256,2496,58044,无供应商信息-商品编码：19013,无,True
3,19013,小象冒泡居家凉拖鞋,19013-015,灰色42-43码,4302,2760,58044,无供应商信息-商品编码：19013,无,True
4,19013,小象冒泡居家凉拖鞋,19013-001,粉色36-37码,3663,1871,58044,无供应商信息-商品编码：19013,无,True
...,...,...,...,...,...,...,...,...,...,...
2605,18315,恶搞圣旨便签本?,18315-003,爱卿平身,2,1,2,无供应商信息-商品编码：18315,无,True
2606,17051,阿狸图案隐形船袜,17051-005,深粉色狐狸,2,-2,2,无供应商信息-商品编码：17051,无,True
2607,18311,中国风塑料圆扇,18311-003,傅朕旨意,2,1,2,无供应商信息-商品编码：18311,无,True
2608,17298,水果半透明、超大笔袋,17298-016,倒梯形-粉色桃子,2,2,2,无供应商信息-商品编码：17298,无,True


In [97]:
len(to3_tem['供应商名称'].unique())  

929

In [98]:
# 并统计出该供应商各商品编码中，销售量最大的一个商品的数量（无供应商信息，则销量就是该商品的销售量）。
# 将供应商信息，销售量最大的商品的数量这两个信息与to2.csv合并，再进行排版规格化，最终形成to3.csv

In [127]:
# t3 =  to3_tem[["商品编码","商品总销量",'供应商名称','供应商旺旺号']]
# t3['供应商名称_'] = t3['供应商名称']

# t33 = t3.groupby(['供应商名称','商品编码']).max()
# # t33 = t3.groupby('供应商名称').agg({'商品编码': np.max})
# t33.rename(columns={'商品总销量': '供应商最大单品销量'}, inplace=True)
# t33


t3 =  to3_tem[["商品编码","商品总销量",'供应商名称','供应商旺旺号']]
t3['供应商名称_'] = t3['供应商名称']

t33 = t3.groupby(['供应商名称','商品编码']).max()
# t33 = t3.groupby('供应商名称').agg({'商品编码': np.max})
t33.rename(columns={'商品总销量': '供应商最大单品销量'}, inplace=True)
t33

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


供应商最大单品销量       供应商旺旺号              供应商名称_
供应商名称              商品编码                                             
上海易旅实业有限公司         16042         47    易旅etravel          上海易旅实业有限公司
上海欣雪货架(多供应商)       仓库用品           5            无        上海欣雪货架(多供应商)
上海联科胶粘制品有限公司       16066        166         联科胶粘        上海联科胶粘制品有限公司
                   16215        466         联科胶粘        上海联科胶粘制品有限公司
上海荷亚装饰材料有限公司(多供应商) 16087         33            无  上海荷亚装饰材料有限公司(多供应商)
...                             ...          ...                 ...
苍南恩赐利文具有限公司        16067       1851      encili3         苍南恩赐利文具有限公司
蔡永红                15098        109       sytonn                 蔡永红
金华市金东区晨星日用品厂(多供应商) 15415        306          傅诗靖  金华市金东区晨星日用品厂(多供应商)
黄岩南城旭日塑料厂          15399       1247  xuriplastic           黄岩南城旭日塑料厂
黄岩南城旭日塑料厂(多供应商)    仓库用品           5  xuriplastic     黄岩南城旭日塑料厂(多供应商)

[946 rows x 3 columns]

In [100]:
# t33 = t33.sort_values(['供应商最大单品销量'], ascending=[False])
# t33

In [128]:
to3 = pd.merge(to2,t33,how='inner',on='商品编码')
to3.rename(columns={'供应商名称_': '供应商名称'}, inplace=True)
to3

# to3 = pd.merge(to2,t33,how='inner',on='供应商名称_')
# # to3.rename(columns={'供应商名称_': '供应商名称'}, inplace=True)
# to3


,商品编码,商品名称,规格编码,规格名称,销售数量,期末库存数量,商品总销量,供应商最大单品销量,供应商旺旺号,供应商名称
0,19013,小象冒泡居家凉拖鞋,19013-005,紫色38-39码,5699,2613,58044,58044,无,无供应商信息-商品编码：19013
1,19013,小象冒泡居家凉拖鞋,19013-012,浅绿38-39码,5259,1898,58044,58044,无,无供应商信息-商品编码：19013
2,19013,小象冒泡居家凉拖鞋,19013-002,粉色38-39码,5256,2496,58044,58044,无,无供应商信息-商品编码：19013
3,19013,小象冒泡居家凉拖鞋,19013-015,灰色42-43码,4302,2760,58044,58044,无,无供应商信息-商品编码：19013
4,19013,小象冒泡居家凉拖鞋,19013-001,粉色36-37码,3663,1871,58044,58044,无,无供应商信息-商品编码：19013
...,...,...,...,...,...,...,...,...,...,...
2605,18315,恶搞圣旨便签本?,18315-003,爱卿平身,2,1,2,2,无,无供应商信息-商品编码：18315
2606,17051,阿狸图案隐形船袜,17051-005,深粉色狐狸,2,-2,2,2,无,无供应商信息-商品编码：17051
2607,18311,中国风塑料圆扇,18311-003,傅朕旨意,2,1,2,2,无,无供应商信息-商品编码：18311
2608,17298,水果半透明、超大笔袋,17298-016,倒梯形-粉色桃子,2,2,2,2,无,无供应商信息-商品编码：17298


In [102]:
# df1.to_csv(path_or_buf=None, sep=’,’, na_rep=”, float_format=None, columns=None, header=True, index=True, index_label=None, mode=’w’, encoding=None)
# path_or_buf：文件名、文件具体、相对路径、文件流等
# sep：文件分割符号
# na_rep：将NaN转换为特定值
# columns：选择部分列写入
# header：忽略列名
# index：False则选择不写入索引

In [129]:
# 根据供应商和编码排序重新排序
to3.sort_values(['商品总销量','商品编码'], ascending=[False, False])
to3

,商品编码,商品名称,规格编码,规格名称,销售数量,期末库存数量,商品总销量,供应商最大单品销量,供应商旺旺号,供应商名称
0,19013,小象冒泡居家凉拖鞋,19013-005,紫色38-39码,5699,2613,58044,58044,无,无供应商信息-商品编码：19013
1,19013,小象冒泡居家凉拖鞋,19013-012,浅绿38-39码,5259,1898,58044,58044,无,无供应商信息-商品编码：19013
2,19013,小象冒泡居家凉拖鞋,19013-002,粉色38-39码,5256,2496,58044,58044,无,无供应商信息-商品编码：19013
3,19013,小象冒泡居家凉拖鞋,19013-015,灰色42-43码,4302,2760,58044,58044,无,无供应商信息-商品编码：19013
4,19013,小象冒泡居家凉拖鞋,19013-001,粉色36-37码,3663,1871,58044,58044,无,无供应商信息-商品编码：19013
...,...,...,...,...,...,...,...,...,...,...
2605,18315,恶搞圣旨便签本?,18315-003,爱卿平身,2,1,2,2,无,无供应商信息-商品编码：18315
2606,17051,阿狸图案隐形船袜,17051-005,深粉色狐狸,2,-2,2,2,无,无供应商信息-商品编码：17051
2607,18311,中国风塑料圆扇,18311-003,傅朕旨意,2,1,2,2,无,无供应商信息-商品编码：18311
2608,17298,水果半透明、超大笔袋,17298-016,倒梯形-粉色桃子,2,2,2,2,无,无供应商信息-商品编码：17298


In [179]:
# 手动拼接成原R语言脚本导出的格式
to3_33 =  to3.groupby(['供应商名称'])
# to3
to3_33

# to3.sort_values(['供应商名称'], ascending=[True])
# to3
res_list = []

obj_empty = {
    '商品编码':'===========',
    '商品名称':'===============================================',
    '规格编码':'===========',
    '规格名称':'===========',
    '销售数量':'===========',
    '期末库存数量':'===========',
    '商品总销售数量':'===========',
    '供应商':'===========',
    '供应商最大单品销量':'===========',
}


for index, value in to3_33:
#     print(value)
    temp = []
    temp.append(obj_empty)

    obj_name = {
        '商品编码':'           ',
        '商品名称':'         ',
        '规格编码':'         ',
        '规格名称':'         ',
        '销售数量':'         ',
        '期末库存数量':'         ',
        '商品总销售数量':'         ',
        '供应商':'         ',
        '供应商最大单品销量':'         ',
    }
    head = index+'   旺旺号： '+value['供应商旺旺号'].values[0] 
    obj_name['商品名称'] = str(head)
    temp.append(obj_name)
    temp.append(obj_empty)
    # 中间插入dataframe的内容
    length = len(value.values.tolist())
    for i in range(0,length):
#         print(value)
#         print(value.iloc[i,'商品编码'])
        obj = {}
        obj['商品编码'] = value.iloc[i,0]
        obj['商品名称'] = value.iloc[i,1]
        obj['规格编码'] = value.iloc[i,2]
        obj['规格名称'] = value.iloc[i,3]
        obj['销售数量'] = value.iloc[i,4]
        obj['期末库存数量'] = value.iloc[i,5]
        obj['商品总销售数量'] = value.iloc[i,6]
        obj['供应商'] = index
        obj['供应商最大单品销量'] = value.iloc[i,7]
        temp.append(obj)
#     temp.append(obj_empty)
    res_list.append(temp)
# print(res_list)
#     print("===========","===============================================","===========","===========","===========","===========","===========","===========","===========")
#     #     print(type(value))  # <class 'pandas.core.frame.DataFrame'>
#     print(index+'   旺旺号： '+value['供应商旺旺号'].values[0],',         ',',         ',',         ',',         ',',         ',',         ',',         ==')
#     print("===========","===============================================","===========","===========","===========","===========","===========","===========","===========")
#     print(value[['商品编码','商品名称','规格编码','规格名称','销售数量','期末库存数量','商品总销量','供应商最大单品销量']])
    


In [180]:
# 排序上面的 [[{},{}],[]]   by 商品总销售数量  0 1 -1 三个不考虑 也可以不用排除 
# res_list
sorted_list = []
for index_out,out in enumerate(res_list):
    for index,inner in enumerate(out):
        if index > 2 and index < len(out) -1:
#             print(index)
            if index == 3:
                temp_dict = {}
                temp_dict['out'] = index_out
                temp_dict['value'] = inner['商品总销售数量']
                sorted_list.append(temp_dict)
sorted_list      
# 重新排序
import operator
sorted_x = sorted(sorted_list, key=operator.itemgetter('value'), reverse=True)  

finall_list = []
for i,item in enumerate(sorted_x):
#     item['new_index'] = i
    finall_list.append(res_list[item['out']])
    

# sorted_x
# finall_list



In [136]:
finall_list # [[],[]]

In [137]:
# 字典变换   写入dataframe效率比较低 时间长
# d_out = []
# for index_out,out in enumerate(finall_list):
#     for inner in out:
#         dd = []
#         dd.append(inner['商品编码'])
#         dd.append(inner['商品名称'])
#         dd.append(inner['规格编码'])
#         dd.append(inner['规格名称'])
#         dd.append(inner['销售数量'])
#         dd.append(inner['期末库存数量'])
#         dd.append(inner['商品总销售数量'])
#         dd.append(inner['供应商'])
#         dd.append(inner['供应商最大单品销量'])
#         d_out.append(dd)
# TODO
# df3.loc['new'] = ['a','a','a','a']

# 写入dataframe效率还是低


In [151]:
# d_out
# finall_list  [[{},{}],[]]  --> 

In [181]:
def parse_finall_list(finall_list):
    v1 = []
    v2 = []
    v3 = []
    v4 = []
    v5 = []
    v6 = []
    v7 = []
    v8 = []
    v9 = []
    for out in finall_list:
        for inner in out:
            v1.append(inner['商品编码'])
            v2.append(inner['商品名称'])
            v3.append(inner['规格编码'])
            v4.append(inner['规格名称'])
            v5.append(inner['销售数量'])
            v6.append(inner['期末库存数量'])
            v7.append(inner['商品总销售数量'])
            v8.append(inner['供应商'])
            v9.append(inner['供应商最大单品销量'])
    
    return v1,v2,v3,v4,v5,v6,v7,v8,v9
        

In [182]:
v1,v2,v3,v4,v5,v6,v7,v8,v9 = parse_finall_list(finall_list)

In [183]:
data = {
    '商品编码':v1,
    '商品名称':v2,
    '规格编码':v3,
    '规格名称':v4,
    '销售数量':v5,
    '期末库存数量':v6,
    '商品总销售数量':v7,
    '供应商':v8,
    '供应商最大单品销量':v9,
}

In [184]:
# 这种写入dataframe的效率比较高
res = pd.DataFrame(data)


In [143]:
# res = pd.DataFrame(columns=["商品编码","商品名称","规格编码","规格名称","销售数量","期末库存数量",'商品总销售数量',"供应商","供应商最大单品销量"])

In [149]:

# # 循环添加数据   too slow
# for index_out,out in enumerate(d_out):
# #     for inner in out:
# #         res = res.append(inner,ignore_index=True)
#     res.loc[index_out] = out
# #     print(out)


In [185]:
res

,商品编码,商品名称,规格编码,规格名称,销售数量,期末库存数量,商品总销售数量,供应商,供应商最大单品销量
0,===========,===============================================,===========,===========,===========,===========,===========,===========,===========
1,,无供应商信息-商品编码：19013 旺旺号： 无,,,,,,,
2,===========,===============================================,===========,===========,===========,===========,===========,===========,===========
3,19013,小象冒泡居家凉拖鞋,19013-005,紫色38-39码,5699,2613,58044,无供应商信息-商品编码：19013,58044
4,19013,小象冒泡居家凉拖鞋,19013-012,浅绿38-39码,5259,1898,58044,无供应商信息-商品编码：19013,58044
...,...,...,...,...,...,...,...,...,...
3864,===========,===============================================,===========,===========,===========,===========,===========,===========,===========
3865,,无供应商信息-商品编码：16358 旺旺号： 无,,,,,,,
3866,===========,===============================================,===========,===========,===========,===========,===========,===========,===========
3867,16358,可爱木质挂钩2只装,16358-005,"木质蓝色花朵,2只装",2,1,4,无供应商信息-商品编码：16358,4


In [ ]:
# 导出csv 即最后的采购决策
res.to_csv('test.csv',index=False)

################### end ##########################